In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.metrics.pairwise import cosine_similarity
import math
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import TweetTokenizer
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)

#sql_select = '''select created_at, location, tweet_content, cleaned_tweet, vector from tweets ORDER BY RANDOM() LIMIT 300000;'''
sql_select = '''select created_at, location, tweet_content, cleaned_tweet, hashtags from tweets;'''
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.reset_index(inplace = True)

In [5]:
len(df)

452102

In [9]:
import pickle
!pip install redis
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')

In [7]:
tweet_tfidf_fit = tfidf_vectorizer.fit(df['cleaned_tweet'].values.astype('U'))
tweet_tfidf_transform = tfidf_vectorizer.fit_transform(df['cleaned_tweet'].values.astype('U'))
tweet_tfidf_fit = tfidf_vectorizer.fit(df['hashtags'].values.astype('U'))
tweet_tfidf_transform = tfidf_vectorizer.fit_transform(df['hashtags'].values.astype('U'))

In [8]:
tweet_tfidf_fit = pickle.dumps(tweet_tfidf_fit)
r.set('tweet_tfidf_fit', tweet_tfidf_fit)
tweet_tfidf_transform = pickle.dumps(tweet_tfidf_transform)
r.set('tweet_tfidf_fit_transform', tweet_tfidf_transform)

True

In [10]:
import pickle
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)
tweet_tfidf = pickle.loads(r.get('tweet_tfidf_fit_transform'))

In [11]:
SVD = TruncatedSVD(n_components = 300)
SVD_model_fit = SVD.fit(tweet_tfidf)
r = redis.StrictRedis(redis_ip)
LSA = pickle.dumps(SVD_model_fit)
r.set('tweet_SVD_fit', LSA)

True

In [12]:
r.keys()

[b'hashtag_mean_arr_window14',
 b'hashtag_mean_arr_window19',
 b'hashtag_mean_arr_window2',
 b'hashtags_tfidf_fit_transform',
 b'hashtag_mean_arr_window23',
 b'hashtag_mean_arr_window5',
 b'tweet_tfidf_fit_transform',
 b'hashtag_mean_arr_window4',
 b'hashtag_mean_arr_window7',
 b'hashtag_mean_arr_window17',
 b'hashtag_mean_arr_window21',
 b'hashtags_countvec_fit_transform',
 b'hashtag_mean_arr_window13',
 b'tweet_SVD_fit',
 b'hashtags_countvec_fit',
 b'hashtag_mean_arr_window15',
 b'hashtag_mean_arr_window6',
 b'hashtag_mean_arr_window12',
 b'hashtag_mean_arr_window10',
 b'hashtag_mean_arr_window3',
 b'hashtag_mean_arr_window8',
 b'hashtag_mean_arr_window11',
 b'hashtag_mean_arr_window16',
 b'hashtag_mean_arr_window20',
 b'hashtag_mean_arr_window18',
 b'hashtag_mean_arr_window9',
 b'tweet_tfidf_fit',
 b'hashtag_mean_arr_window0',
 b'hashtag_mean_arr_window22',
 b'hashtag_mean_arr_window1']